In [1]:
import re

dict_list = {}
idx_list = {}

file = open('10.warc_index.dict', 'r')
line = file.readlines()
for d in line:
    d_split = d.split(", ")
    dict_list[d_split[0]] = {}
    dict_list[d_split[0]]["text"] = d_split[1].strip()
file.close()

file = open('10.warc_index.idx', 'r')
line = file.readlines()
for d in line:
    d_split = d.split(", ", 1)
    idx_list[d_split[0]] = {}
    idx_list[d_split[0]]["text"] = d_split[1].strip()
file.close()

In [2]:
import math
import pandas as pd
# 40610 maybe total
word = {}
q_ans = {}
d_ans = {}
query = input()

# query term frequency
for q in query.split():
    if q in word:
        word[q] = word[q] + 1
    else:
        word[q] = 1

for q in query.split():
    tf = {}
    if q in dict_list:
#         idf
        dict_list[q]["idf"] = math.log(40610/int(dict_list[q]["text"]))
        
#         doc tfidf
        idx = re.split(r':<|>;', idx_list[q]["text"])
        for t in range(1, len(idx)-3, 2):
            t_split = idx[t].split(", ")
            tf[t_split[0]] = (1 + math.log(int(t_split[1]))) * dict_list[q]["idf"]
        d_ans[q] = tf 
        
#         query tfidf
        q_ans[q] = {}
        q_ans[q]["q_tfidf"] = (1 + math.log(word[q])) * dict_list[q]["idf"]

d_df = pd.DataFrame(d_ans).fillna(0).T
q_df = pd.DataFrame(q_ans).fillna(0).T

AAAAA AAAA


In [3]:
each_sim = []
for t in d_df.columns:
    dot_product = (d_df[t] * q_df['q_tfidf']).sum()
    doc_V = math.sqrt((d_df[t] * d_df[t]).sum())
    query_V = math.sqrt((q_df * q_df).sum())
    
    if doc_V == 0 or query_V == 0:
        each_sim.append(0)
    else:
        each_sim.append(round(dot_product / (doc_V * query_V), 12))
df_sim = pd.DataFrame(each_sim, columns = ['cosine-similarity'])
df_sim.index = d_df.columns

In [4]:
sort_ans = df_sim['cosine-similarity'].sort_values(ascending = False).reset_index()
sort_ans.columns = ['document', 'cosine-similarity']
sort_ans.to_csv('vsm.result.csv', index=0)